###런타임 아웃 방지 (https://dacon.io/forum/405904)
function KeepClicking(){

console.log("Clicking");

document.querySelector("colab-connect-button").click()

}

setInterval(KeepClicking,60000)

# **import data**

In [1]:
import pandas as pd

from google.colab import files
uploaded = files.upload()
#Air_reviews.xlsx

Saving Air_reviews.xlsx to Air_reviews.xlsx


In [3]:
df = pd.read_excel('Air_reviews.xlsx')
df.head()

,reviews,dfake
0,"LG 공기청정기 고민했는데, 구매하고 나서 너무 다행이에요. 미세먼지가 확실히 줄어...",1
1,LG 공기청정기 구매했는데 정말 생각보다 성능이 좋네요! 미세먼지가 제거되니 공기 ...,1
2,"LG 공기청정기 구매했는데, 미세먼지 제거가 뛰어나네요. 생각보다 효과가 좋아서 매...",1
3,"LG 공기청정기 구매했는데, 생각보다 성능이 뛰어나요. 미세먼지가 확실히 줄어들고,...",1
4,"LG 공기청정기 구매했는데, 이미 사용 중인 삼성 제품과 비교해도 성능이 뛰어나네요...",1


# **전처리**

In [5]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-vxhdmaky
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-vxhdmaky
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit b32a889cbd10b006d2f4aba118f0cd5b677e2979
  Preparing metadata (setup.py) ... done
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2286920 sha256=72c2e553b6fb3b6888160c150bf3ae9691fd6152fdd8ac38781414ce71e4bcad
  Stored in directory: /tmp/pip-ephem-wheel-cache-pyjngrfu/wheels/76/b3/33/dda14886ee76b8e53eb05580a14dfcf9145e9eb9d282c53f28
Successfully built pykospacing


In [7]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 29.9 MB/s eta 0:00:00


In [8]:
#import할 패키지 목록
import re
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
from pykospacing import Spacing
from collections import Counter


In [9]:
#null값 확인
df.isnull().sum()

,0
reviews,0
dfake,0


In [12]:
# 10자 이상 중복된 행 필터링 및 삭제
duplicates = df[df['reviews'].duplicated(keep=False)]
to_drop = duplicates[duplicates['reviews'].str.len() >= 10].index
reviews_air = df.drop(to_drop)

# 인덱스 재배열
reviews_air = reviews_air.reset_index(drop=True);reviews_air

,reviews,dfake
0,"LG 공기청정기 고민했는데, 구매하고 나서 너무 다행이에요. 미세먼지가 확실히 줄어...",1
1,LG 공기청정기 구매했는데 정말 생각보다 성능이 좋네요! 미세먼지가 제거되니 공기 ...,1
2,"LG 공기청정기 구매했는데, 미세먼지 제거가 뛰어나네요. 생각보다 효과가 좋아서 매...",1
3,"LG 공기청정기 구매했는데, 생각보다 성능이 뛰어나요. 미세먼지가 확실히 줄어들고,...",1
4,"LG 공기청정기 구매했는데, 이미 사용 중인 삼성 제품과 비교해도 성능이 뛰어나네요...",1
...,...,...
6827,배송도 빠르고 가격도 저렴히 판매해주셔서 너무 만족합니다제품 하자없이 뛰어난 성능으...,0
6828,배송비는 바로 받아놓고 계속 전화도 안받으셨지만... 배송믄 2주 조금 더 걸렸습니...,0
6829,배송도 빠르고 친절하였습니다,0
6830,공기청정기는 lg제품이 최고입니다적당한가격에 구매하여 아껴가며 오래도록 쓰겠 습니다,0


In [14]:
#한글표현만 남기기
def extract_word(text):
    hangul = re.compile('[^가-힣]')
    result = hangul.sub(' ', text)
    return result
reviews_air['reviews'] = reviews_air['reviews'].apply(lambda x:extract_word(x))
reviews_air

,reviews,dfake
0,공기청정기 고민했는데 구매하고 나서 너무 다행이에요 미세먼지가 확실히 줄어...,1
1,공기청정기 구매했는데 정말 생각보다 성능이 좋네요 미세먼지가 제거되니 공기 ...,1
2,공기청정기 구매했는데 미세먼지 제거가 뛰어나네요 생각보다 효과가 좋아서 매...,1
3,공기청정기 구매했는데 생각보다 성능이 뛰어나요 미세먼지가 확실히 줄어들고 ...,1
4,공기청정기 구매했는데 이미 사용 중인 삼성 제품과 비교해도 성능이 뛰어나네요...,1
...,...,...
6827,배송도 빠르고 가격도 저렴히 판매해주셔서 너무 만족합니다제품 하자없이 뛰어난 성능으...,0
6828,배송비는 바로 받아놓고 계속 전화도 안받으셨지만 배송믄 주 조금 더 걸렸습니...,0
6829,배송도 빠르고 친절하였습니다,0
6830,공기청정기는 제품이 최고입니다적당한가격에 구매하여 아껴가며 오래도록 쓰겠 습니다,0


In [15]:
#띄어쓰기 수정
spacing = Spacing()
# reviews_air['reviews']에 spacing 적용
reviews_air['reviews'] = reviews_air['reviews'].apply(spacing)

In [32]:
X_data = reviews_air['reviews']
Y_data = reviews_air['dfake']

In [33]:
# 형태소 분석
okt = Okt()
words = " ".join(X_data.tolist())
words = okt.morphs(words, stem=True)


In [34]:
#불용어 제거

remove_one_word = [x for x in words if len(x)>1]

len(remove_one_word) #개수

88115

In [35]:
#collections의 Counter객체를 이용하여 단어별 빈도를 확인
from collections import Counter
frequent = Counter(remove_one_word).most_common();frequent

[('하다', 5263),
 ('좋다', 3878),
 ('자다', 2209),
 ('있다', 1835),
 ('배송', 1737),
 ('사용', 1504),
 ('공기청정기', 1210),
 ('제품', 1133),
 ('너무', 1007),
 ('공기', 1005),
 ('빠르다', 1004),
 ('보다', 988),
 ('구매', 980),
 ('성능', 951),
 ('디자인', 918),
 ('되다', 908),
 ('같다', 877),
 ('정말', 822),
 ('만족하다', 750),
 ('가격', 741),
 ('소음', 720),
 ('으로', 702),
 ('필터', 684),
 ('없다', 678),
 ('생각', 658),
 ('이다', 619),
 ('가성', 596),
 ('하고', 575),
 ('크다', 574),
 ('받다', 557),
 ('조용하다', 530),
 ('설치', 507),
 ('미세먼지', 474),
 ('쓰다', 474),
 ('작동', 468),
 ('사다', 449),
 ('에서', 408),
 ('저렴하다', 381),
 ('삼성', 374),
 ('구입', 366),
 ('추천', 365),
 ('아주', 365),
 ('않다', 363),
 ('기능', 352),
 ('들다', 352),
 ('깔끔하다', 339),
 ('친절하다', 317),
 ('닉스', 312),
 ('냄새', 304),
 ('마음', 302),
 ('걱정', 302),
 ('확실하다', 295),
 ('간편하다', 292),
 ('되어다', 288),
 ('만족스럽다', 287),
 ('크기', 276),
 ('감사하다', 271),
 ('뛰어나다', 268),
 ('거실', 267),
 ('편하다', 263),
 ('해주다', 262),
 ('오다', 258),
 ('편리하다', 251),
 ('대비', 250),
 ('이라', 244),
 ('어울리다', 243),
 ('모드', 242),
 ('기사', 238),
 (

In [19]:
# stopwords(불용어) 파일 업로드
from google.colab import files
uploaded = files.upload()

Saving stopwords.txt to stopwords.txt


In [36]:
with open('stopwords.txt', 'r', encoding='cp949') as f:
    list_file = f.readlines()
stopwords = list_file[0].split(",")
remove_stopwords = [x for x in remove_one_word if x not in stopwords]
len(remove_stopwords)
#불용어 제거 후 8만 -> 6만

62911

In [37]:
Counter(remove_stopwords).most_common()
#이전보다 의미있는 단어만 남음

[('좋다', 3878),
 ('자다', 2209),
 ('배송', 1737),
 ('사용', 1504),
 ('공기청정기', 1210),
 ('제품', 1133),
 ('공기', 1005),
 ('빠르다', 1004),
 ('구매', 980),
 ('성능', 951),
 ('디자인', 918),
 ('만족하다', 750),
 ('가격', 741),
 ('소음', 720),
 ('필터', 684),
 ('생각', 658),
 ('가성', 596),
 ('크다', 574),
 ('조용하다', 530),
 ('설치', 507),
 ('미세먼지', 474),
 ('쓰다', 474),
 ('작동', 468),
 ('사다', 449),
 ('저렴하다', 381),
 ('삼성', 374),
 ('구입', 366),
 ('추천', 365),
 ('아주', 365),
 ('기능', 352),
 ('깔끔하다', 339),
 ('친절하다', 317),
 ('닉스', 312),
 ('냄새', 304),
 ('마음', 302),
 ('걱정', 302),
 ('확실하다', 295),
 ('간편하다', 292),
 ('만족스럽다', 287),
 ('크기', 276),
 ('감사하다', 271),
 ('뛰어나다', 268),
 ('거실', 267),
 ('편하다', 263),
 ('해주다', 262),
 ('편리하다', 251),
 ('대비', 250),
 ('어울리다', 243),
 ('모드', 242),
 ('기사', 238),
 ('관리', 233),
 ('청정', 227),
 ('상품', 221),
 ('아기', 217),
 ('괜찮다', 217),
 ('좋아지다', 212),
 ('선택', 206),
 ('예쁘다', 206),
 ('최고', 203),
 ('집안', 200),
 ('고민', 189),
 ('에도', 186),
 ('만족', 186),
 ('느낌', 185),
 ('믿다', 180),
 ('주문', 175),
 ('놓다', 174),
 ('해보다', 173),
 

# **1. 로지스틱 회귀 분석**

In [22]:

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

# X_data를 불용어 제거 후, 리뷰별 텍스트 리스트로 변환
processed_reviews = [" ".join([word for word in okt.morphs(review, stem=True) if word not in stopwords])
                     for review in X_data]

# 벡터화 (리뷰별로 각 레코드를 벡터화함)
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_vectorized = tfidf_vectorizer.fit_transform(processed_reviews)

In [45]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_vectorized, Y_data, test_size=0.4, random_state=42)


In [46]:

# 로지스틱 회귀 모델 정의 및 학습
model = LogisticRegression()
model.fit(X_train, y_train)

# 예측 수행
y_pred = model.predict(X_test)

In [47]:
# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.9370654957921698
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.97      0.96      1901
           1       0.93      0.86      0.89       832

    accuracy                           0.94      2733
   macro avg       0.93      0.92      0.92      2733
weighted avg       0.94      0.94      0.94      2733



In [50]:
for C_value in [0.01, 0.1, 1, 10, 100]:
    model_rt = LogisticRegression(C=C_value)
    model_rt.fit(X_train, y_train)
    y_pred_rt = model_rt.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_rt)
    print(f"C = {C_value}, Accuracy = {accuracy}")

C = 0.01, Accuracy = 0.6955726308086352
C = 0.1, Accuracy = 0.8693743139407245
C = 1, Accuracy = 0.9370654957921698
C = 10, Accuracy = 0.9458470545188438
C = 100, Accuracy = 0.9421880717160629


# **2. 서포트벡터머신**

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

# X_data를 불용어 제거 후, 리뷰별 텍스트 리스트로 변환
processed_reviews = [" ".join([word for word in okt.morphs(review, stem=True) if word not in stopwords])
                     for review in X_data]

# 벡터화 (리뷰별로 각 레코드를 벡터화함)
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_vectorized = tfidf_vectorizer.fit_transform(processed_reviews)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_vectorized, Y_data, test_size=0.4, random_state=42)


In [53]:
# SVM 모델 정의 및 학습
model = SVC(kernel='linear')  # 선형 커널을 사용한 SVM
model.fit(X_train, y_train)

# 예측 수행
y_pred = model.predict(X_test)


In [54]:
# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.9480424442005122
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.96      0.96      1901
           1       0.92      0.91      0.91       832

    accuracy                           0.95      2733
   macro avg       0.94      0.94      0.94      2733
weighted avg       0.95      0.95      0.95      2733



In [56]:
from sklearn.svm import SVC

# SVM 모델 정의 (C 값을 설정하여 규제 강도 조절)
for C_value in [0.01, 0.1, 1, 10, 100]:
    model_rt = SVC(kernel = 'linear', C=C_value)
    model_rt.fit(X_train, y_train)
    y_pred_rt = model_rt.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_rt)
    print(f"C = {C_value}, Accuracy = {accuracy}")

C = 0.01, Accuracy = 0.6955726308086352
C = 0.1, Accuracy = 0.9268203439443835
C = 1, Accuracy = 0.9480424442005122
C = 10, Accuracy = 0.9363336992316136
C = 100, Accuracy = 0.933772411269667


# **3. GBM(그래디언트 부스트 머신)**

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

# X_data를 불용어 제거 후, 리뷰별 텍스트 리스트로 변환
processed_reviews = [" ".join([word for word in okt.morphs(review, stem=True) if word not in stopwords])
                     for review in X_data]

# 벡터화 (리뷰별로 각 레코드를 벡터화함)
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_vectorized = tfidf_vectorizer.fit_transform(processed_reviews)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_vectorized, Y_data, test_size=0.4, random_state=42)


In [59]:
# GBM 모델 정의 및 학습
model = GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)

# 예측 수행
y_pred = model.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.924624954262715
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.98      0.95      1901
           1       0.93      0.81      0.87       832

    accuracy                           0.92      2733
   macro avg       0.93      0.89      0.91      2733
weighted avg       0.93      0.92      0.92      2733



In [60]:
from sklearn.ensemble import GradientBoostingClassifier

# 학습률과 트리 개수의 다양한 옵션을 설정
learning_rates = [0.01, 0.1, 0.2]
n_estimators_options = [50, 100, 200]

# 민감도 분석 결과를 저장할 리스트
results = []

# 반복문을 통해 다양한 learning_rate와 n_estimators에 대한 성능을 평가
for lr in learning_rates:
    for n_est in n_estimators_options:
        model = GradientBoostingClassifier(learning_rate=lr, n_estimators=n_est, random_state=42)
        model.fit(X_train, y_train)

        # 예측 수행
        y_pred = model.predict(X_test)

        # 정확도 계산
        accuracy = accuracy_score(y_test, y_pred)

        # 결과 저장
        results.append({
            'learning_rate': lr,
            'n_estimators': n_est,
            'accuracy': accuracy
        })

# 결과 출력
results_df = pd.DataFrame(results)
print(results_df)

   learning_rate  n_estimators  accuracy
0           0.01            50  0.725942
1           0.01           100  0.801683
2           0.01           200  0.856934
3           0.10            50  0.902671
4           0.10           100  0.924625
5           0.10           200  0.939993
6           0.20            50  0.929016
7           0.20           100  0.942188
8           0.20           200  0.945481
